# Impordi ja defineeri vajalikud meetodid

In [18]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import re
import os


def make_splits(start, interval, time_before, time_after, measurements_count):
    splits = []
    if(measurements_count>0):
        splits.append((start - timedelta(seconds=time_before), start + timedelta(seconds=time_after)))
        splits.extend(make_splits(start + timedelta(seconds=interval),interval, time_before, time_after, measurements_count-1))
    return splits

def assign_range(times, splits):
    c_ix = 0
    in_split = False
    ranges = []
    for index, time in times.iteritems():
        if(splits[c_ix][0] <= time <= splits[c_ix][1]):
            in_split = True
        elif(in_split == True):
            if(c_ix +1 < len(splits)):
                c_ix += 1
            in_split = False

        if(in_split):
            ranges.append(str(splits[c_ix]))
        else:
            ranges.append(None)
    ranges = np.array(ranges)
    return ranges

def window_rms(a, window_size):
    a2 = np.power(a,2)
    window = np.ones(window_size)/float(window_size)
    return np.sqrt(np.convolve(a2, window, 'valid'))


# Defineeri muutujad

In [ ]:
recordings_file = './Recordings/17. January/Algus_17_33_01.txt'
target_file = './Recordings/17. January/vastused_hain.txt'

start_datetime = datetime(year=2020, month=1, day=17, hour=17, minute=33, second=0)
interval = 10
time_before = 1
time_after = 3
measurements_count = 60


# Data preprocessing

In [2]:
data = pd.read_csv(recordings_file, skiprows=6, delimiter=",", names=np.arange(13))
data = data.reset_index()
data['datetime'] = data[12].apply(lambda x: datetime.fromtimestamp(x/1000.0))

#siin vaata, mis kanalil brow ja mis kanalil cheek oli
data = data[[2,3,5,'datetime']]
data.columns = ["unrailed","brow", "cheek", "datetime"]

### Assign timestamps into ranges
splits = make_splits(start_datetime, interval, time_before, time_after, measurements_count)
splits[:5]
data['ranges'] = assign_range(data['datetime'], splits)

groups = data.groupby('ranges')
groups_min = groups.size().min()

cheek_array = groups['cheek'].apply(lambda x: x.head(groups_min)).values
cheek_array = cheek_array.reshape(len(groups), groups_min)

brow_array = groups['brow'].apply(lambda x: x.head(groups_min)).values
brow_array = cheek_array.reshape(len(groups), groups_min)

with open(target_file, 'r') as file:
    lines = file.readlines()[1:]    

pattern = r'(\D)\s(\D)\n'
joined_lines = "\n".join(lines)
meta = re.findall(pattern,joined_lines)
meta = np.array([list(m)for m in meta])
meta = meta.T

#truth or false
decision = meta[0]

# yes or no
action = meta[1]

meta[0] = meta[0] == 'T'
meta[1] = meta[1] == 'J'


# Save files

In [64]:
file_count = len(os.listdir('./Processed_data/Cheek/'))

with open("./Processed_data/Cheek/{}".format(file_count+1), 'wb') as file:
    np.save(file, cheek_array)
    
with open("./Processed_data/Brow/{}".format(file_count+1), 'wb') as file:
    np.save(file, brow_array)
    
with open("./Processed_data/Decisions/{}".format(file_count+1), 'wb') as file:
    np.save(file, meta)
    

# Read in files

In [65]:
import numpy as np

#specify, what file nr to read
file_nr = 1

with open("./Processed_data/Cheek/{}".format(file_nr), 'rb') as file:
    cheek_array = np.load(file)
    
with open("./Processed_data/Brow/{}".format(file_nr), 'rb') as file:
    brow_array = np.load(file)
    
with open("./Processed_data/Decisions/{}".format(file_nr), 'rb') as file:
    meta = np.load(file)
    